# Chess AI: MiniMax Algorithm

### Packages

In [2]:
## math, randomization
import numpy as np
import random

## copy board state prior to making move
from copy import deepcopy

## will need to keep track of time
import time

## library for chess UI
import chess

### Coding the Chess Bot

Step 1: Define heuristic function(s) for the agent to use when evaluating a given game state. 

In [7]:
class myEval_Material: 

    def score(self, board, turn=chess.WHITE): 

        """ return a score for the given game state. 
        1. create dictionary of piece key: values
        2. multiply dictionary by 1 or -1, depending on turn
        3. retrieve distribution of pieces on board
        4. sum values for these pieces
        """

        # 1. create dictionary of piece key: value pairs (white first)

        white_scoring = {
            'p': -1,
            'n': -3,
            'b': -3,
            'r': -5,
            'q': -9,
            'k': 0,
            'P': 1,
            'N': 3,
            'B': 3,
            'R': 5,
            'Q': 9,
            'K': 0,
        }

        # 2. create negated version for black

        black_scoring = {key: value * -1 for key, value in white_scoring.items()}

        # 3. retrieve distribution of pieces on board

        pieces = board.piece_map()

        # 4. sum values for pieces on board

        if turn == chess.WHITE:
            scoring = white_scoring
        else:
            scoring = black_scoring
        
        score = 0
        for key in pieces:
            score += scoring[str(pieces[key])]

        # output final score
        return score


Step 2: Create class to represent chess player. details below

In [9]:
class myChessPlayer:

    """ Intelligent chess agent based on minimax algorithm. 
    DETAILS: 
    - Chess bot that chooses a move using a custom evaluation (i.e., heuristic) function. 
    - Utilizes the minimax algorithm with the following optimizations:
      - alpha beta pruning
      - iterative deepening 
    IMPLEMENTATION STEPS
    - 
    """
    
    def __init__(self, depth=50, eval=myEval_Material()): 
        
        """Initializes your player.
        Args: 
        - depth (int): depth of tree to which your agent will search
        - eval (function): evaluation function used by your agent
        """

        self.depth = depth
        self.eval = eval

    def move(self, game, time_left): 
        
        """Called to determine one move by your agent
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: (int,int): Your best move
        """
        best_move, utility = my_minimax(self, game, time_left, depth=self.depth)
        return best_move
    
    def utility(self, game, my_turn, is_over=False, win=False):
        """You can handle special cases here (e.g. endgame)"""
        if is_over:
            if win:
                return 1000
            else:
                return -1000
        return self.eval.score(game, self)

Step 3: Define minimax function

In [ ]:
def my_minimax(player, game, time_left, depth, alpha=float("-inf"), beta=float("inf"), my_turn=True, time_limit=0.97):
    """Implementation of the alphabeta algorithm.if is_over:
            if win:
                return 1000
            else:
                return -1000

    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer()
            that represents your agent. It is used to call anything you need
            from the CustomPlayer class (the utility() method, for example,
            or any class variables that belong to CustomPlayer())
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        alpha (float): Alpha value for pruning
        beta (float): Beta value for pruning
        my_turn (bool): True if you are computing scores during your turn.

    Returns:
        (tuple, int): best_move, val
    """

    start_time = time.time()

    my_states = set()
    my_states_dict = {}

    their_states = set()
    their_states_dict = {}

    def search_ab(game, d, alpha, beta, my_turn):

        if time.time() - start_time >= time_limit:
            return (None, -1000000000)

        if my_turn:
            best_move, best_score = max_ab(game, d, alpha, beta)
        else:
            best_move, best_score = min_ab(game, d, alpha, beta)

        return (best_move, best_score)

    def max_ab(game, d, alpha, beta):

        ## maximize on player's turn
        possible_moves = list(game.generate_legal_moves())

        ## tracker variables
        best_move = None
        best_score = -1000000

        for move in possible_moves:

            new_game, is_over, _ = game.forecast_move(move)

            if new_game in my_states:
                score = my_states_dict[move]

            ## if move results in game over, or if depth limit has been reached, evaluate game state
            elif is_over or d == 1:
                score = player.utility(new_game, my_turn, is_over, True)

            ## if game is in progress, call min function
            else:
                _, score = search_ab(new_game, d-1, alpha, beta, my_turn=False)

            if new_game not in my_states:
                my_states.add(move)
                my_states_dict[move] = score

            ## alpha-beta pruning
            if score >= beta:
                return (move, score)

            if score > best_score:
                best_score = score
                best_move = move
                alpha = max(score, alpha)

        return (best_move, best_score)

    def min_ab(game, d, alpha, beta):

        ## minimize on opponent's turn
        possible_moves = game.get_opponent_moves(player)

        ## tracker variables
        best_move = None
        best_score = 1000000

        for move in possible_moves:

            new_game, is_over, _ = game.forecast_move(move)

            if new_game in their_states:
                score = their_states_dict[move]

            ## if game is over, evalate game state
            elif is_over or d == 1:
                score = player.utility(new_game, my_turn, is_over, False)

            ## if game is in progress, call max function
            else:
                _, score = search_ab(new_game, d-1, alpha, beta, my_turn=True)

            if new_game not in their_states:
                their_states.add(new_game)
                their_states_dict[new_game] = score

            ## alpha-beta pruning
            if score <= alpha:
                return (move, score)

            if score < best_score:
                best_score = score
                best_move = move
                beta = min(score, beta)

        return (best_move, best_score)

    final_move = None
    final_score = -10000000

    for d in range(3, depth+1):
        if time.time() - start_time >= time_limit:
            break
        for move in game.get_player_moves(player):
            move, score = search_ab(game, d, alpha, beta, my_turn=True)
            if score > final_score:
                final_move = move
                final_score = score

    return (final_move, final_score)

### Testing

In [8]:
my_game = chess.Board()

my_eval = myEval_Material()

my_eval.score(my_game)

0

### OLD WORK

In [34]:
minimax(game, chess.BLACK)


Move.from_uci('g8h6')

In [33]:
game.push_san('g1h3')

Move.from_uci('g1h3')

In [35]:
def minimax_N(board, turn):

    moves = list(board.generate_legal_moves())
    scores = []
    for move in moves:
        test_board = deepcopy(board)
        scores.append(eval_board(test_board, turn))

    best_move = moves[np.argmax(scores)]

    if N>1:

        board.push(best_move)
        temp_best_move = minimax_N(board)
    
    return(best_move)
